<a href="https://colab.research.google.com/github/MOOwuttichai/Cancer_Social_Listening/blob/main/Copy_of_%E0%B8%9A%E0%B8%AD%E0%B8%81%E0%B9%82%E0%B8%A3%E0%B8%84%2B%E0%B9%80%E0%B8%9E%E0%B8%A8%2B%E0%B9%83%E0%B8%84%E0%B8%A3%E0%B9%80%E0%B8%9B%E0%B9%87%E0%B8%99%E0%B8%84%E0%B8%99%E0%B9%80%E0%B8%A5%E0%B9%88%E0%B8%B2%2BSentenceTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install attacut
!pip install pythainlp
!pip install epitran
!pip install -U sentence-transformers

Exception in thread Thread-5 (attachment_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 237, in listen
    sock, _ = endpoints_listener.accept()
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = self._accept()
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 52, in attachment_entry
    debugpy.listen(_dap_port)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/public_api.py", line 31, in wrapper
    return wrapped(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 143, in debug
    log.reraise

In [2]:
import pandas as pd
from google.colab import drive
from attacut import tokenize, Tokenizer
import pythainlp
from pythainlp import sent_tokenize, word_tokenize, Tokenizer
from pythainlp.util import normalize
from pythainlp.corpus.common import thai_words
from collections import OrderedDict
import matplotlib as mpl
import numpy as np
from sklearn.cluster import KMeans
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Carbon/data/data_commentsFB_docter.csv')

In [ ]:
data.rename(columns={'name': 'ชื่อ', 'comments': 'คำพูดโรค'}, inplace=True)

In [ ]:
data

In [ ]:
comment=data.groupby('ชื่อ').sum().reset_index()

In [ ]:
words = set(thai_words())
words.remove("โรคมะเร็ง")
name =['กระเพาะปัสสวะ','กระเพาะปัสสาวะ','เยื่อบุโพรงมดลูก','ปากมดลูก','เม็ดเลือดขาว','กระเพาะอาหาร','กระเพราะอาหาร','ต่อมไทรอยด์','ต่อมไทยรอยด์','ท่อน้ำดี']
for i in name:
  words.add(i)
custom_tokenizer = Tokenizer(words)

In [ ]:
comment['คำพูดโรค'][0]

In [ ]:
list_token =[]
for i in range(len(comment)):#len(comment)
  text= comment['คำพูดโรค'][i]
  custom_tokenizer = Tokenizer(words)
  Token = custom_tokenizer.word_tokenize(normalize(str(text)))
  Token.append('end')
  list_token.append(Token)

#Facebook

##หาโรค

In [ ]:
name_cancar = ['มะเร็งกระเพาะปัสสวะ','มะเร็งเยื่อบุโพรงมดลูก','มะเร็งปากมดลูก','มะเร็งเม็ดเลือดขาว','มะเร็งตับ','มะเร็งตับอ่อน','มะเร็งต่อมลูกหมาก','มะเร็งกระเพาะปัสสาวะ','มะเร็งกระเพาะอาหาร'
,'มะเร็งหลอดอาหาร','มะเร็งเต้านม','มะเร็งปอด','มะเร็งลำไส้ใหญ่','มะเร็งรังไข่','มะเร็งช่องปาก','มะเร็งต่อมไทรอยด์','มะเร็งต่อมน้ำเหลือง','มะเร็งสมอง','มะเร็งผิวหนัง','มะเร็งลำไส้'
,'มะเร็งท่อน้ำดี',"มะเร็งลำไส้ใหญ่และทวารหนัก","มะเร็งผิวหนังชนิดเมลาโนมา","มะเร็งต่อมน้ำเหลืองชนิดนอนฮอดจ์กิน","มะเร็งเม็ดเลือดขาวชนิดเฉียบพลัน","มะเร็งกระดูก","มะเร็งสมองและระบบประสาทกลาง","มะเร็งไต",
"มะเร็งมดลูก","มะเร็งถุงน้ำดี","มะเร็งกระดูกสันหลัง","มะเร็งลูกตา","มะเร็งเยื่อบุช่องคลอด","มะเร็งเยื่อบุอวัยวะเพศชาย","มะเร็งเยื่อบุอวัยวะเพศหญิง","มะเร็งต่อมหมวกไต","มะเร็งต่อมน้ำลาย","มะเร็งถุงน้ำรังไข่"
]#อันนี้ prom ของ gimini "จินี ช่วยสร้างlist python โดยในlistนั้นมีการระบุโรคมะเร็งชนิดต่างๆให้หน่อยขอเป็นเเบบภาษาทางการเเละก็ภาษาพูด ขอเเบบไม่ซ้ำด้วย"

In [ ]:
#หาโรค
#--------------------------------------------------------
new_colcan = []
for i in range(len(list_token)):#len(comment)
  list_cancer = []
  for k in range(len(list_token[i])):
    if (list_token[i][k] == "มะเร็ง")|(list_token[i][k] == "โรคมะเร็ง"):
      list_cancer.append(list_token[i][k]+list_token[i][k+1])
    # else :
    #   list_cancer.append('ไม่สามารถระบุได้')
  unique_list = list(OrderedDict.fromkeys(list_cancer))
  #----------------------------------------------------------
  list_define_cancer = []
  new_list=[]
  for i in range(len(unique_list)):
    for j in range(len(name_cancar)):
      if unique_list[i]==name_cancar[j]:
        list_define_cancer.append(unique_list[i])
  #-----------------------------------------------------
  cancer_list_de2 =[]
  if len(list_define_cancer) > 0:
    if len(list_define_cancer) == 2:
      cancer_list_de2.append('เล่ามากกว่า 2 โรค')
    elif len(list_define_cancer)==1:
      cancer_list_de2.append(list_define_cancer[0])
  elif list_define_cancer==[]:
    cancer_list_de2.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
  if len(cancer_list_de2)> 0 :
    new_colcan.append(cancer_list_de2[0])
  elif len(cancer_list_de2)== 0 :
    new_colcan.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
print(new_colcan)

##หาเพศ

In [ ]:
Genden = {'ชาย':['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ',
                 'ปู่ทวด','ตาทวด','ลุง','อาหนู','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย',
                 'พ่อตา','พ่อผม','พ่อหนู','พ่อพม','ชาย','หนุ่ม','ลช.','ผ่อ','ชย.','น้าชาย','ผ่อตา','หน.']
          ,'หญิง':['แม่','เเม่','คุณแม่','มารดา','พี่สาว','น้องสาว','ลูกสาว','ภรรยา','เมีย','ย่า','ยาย','คุณย่า',
                   'คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า','น้า','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้',
                   'ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้','เเม่ผม','เเม่หนู','เเม่พม','แม่ผม','แม่หนู','แม่พม','สาว','หญิง','ก้อน','คลำ']}

In [ ]:
len(Genden['ชาย'])

In [ ]:
new_colgenden=[]
list_genden=[]
for i in range(len(list_token)):
  for j in range(len(Genden['หญิง'])):
    for k in range(len(list_token[i])):
      if (list_token[i][k] == Genden['ชาย'][j]):
        list_genden.append('เพศชาย')
      elif(list_token[i][k] == Genden['หญิง'][j]):
        list_genden.append('เพศหญิง')
  genden_list =[]
  genden_list = list(OrderedDict.fromkeys(list_genden)) # ลบคำซ้ำ
  #-------------------------------------------------------------------
  list_define_genden = []
  if len(genden_list) > 0 :
    if len(genden_list) == 2:
      list_define_genden.append('เล่าทั้งสองเพศ')
    elif len(genden_list)==1:
      list_define_genden.append(genden_list[0])
  elif len(genden_list)==0:
    list_define_genden.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
  genden_list_de =[]
  genden_list_de = list(OrderedDict.fromkeys(list_define_genden))
  new_colgenden.append(genden_list_de[0])
print(new_colgenden)

In [ ]:
comment

In [ ]:
point =[]
k =[]
r=comment
for i in range(len(list_token)):
  count_text = len(list_token[i])-1
  if new_colcan[i] =='ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น':
    x=0
  elif new_colcan[i] == 'เล่ามากกว่า 2 โรค':
    x=1
  else :
    x=2
  point.append(count_text+x)
  pop = point[i]/100
  if pop >= 0.75:
    k.append('มีประโยชน์มาก')
  elif pop < 0.75 and pop >=0.5:
    k.append('มีประโยชน์')
  elif pop < 0.5 and pop >=0.25:
    k.append('อาจมีประโยชน์')
  elif pop < 0.25 :
    k.append('ไม่มีประโยชน์')
# elif max(point) > 100:
#   for i in range(len(comment)):
#     pop = point[i]/max(point)
#     if pop >= 0.75:
#       k.append('มีประโยชน์มาก')
#     elif pop < 0.75 and pop >=0.5:
#       k.append('มีประโยชน์')
#     elif pop < 0.5 and pop >=0.25:
#       k.append('อาจมีประโยชน์')
#     elif pop < 0.25 :
#       k.append('ไม่มีประโยชน์')
r['โรค'] = new_colcan
r['คะเเนน'] = point
r['ผลลัพธ์'] = k
print(r)

In [ ]:
r

In [ ]:
def detect_person(comment):
    # คำที่ใช้ตรวจสอบว่ามีใครเป็นคนอยู่ในความคิดเห็น
    other = ['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ',
              'ปู่ทวด','ตาทวด','ลุง','อาหนู','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย',
              'พ่อตา','พ่อผม','พ่อหนู','พ่อพม','ชาย','หนุ่ม''แม่','เเม่','คุณแม่','มารดา','พี่สาว','น้องสาว',
              'ลูกสาว','ภรรยา','เมีย','ย่า','ยาย','คุณย่า','คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า',
              'น้า','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้','ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้','เเม่ผม','เเม่หนู','เเม่พม','แม่ผม','แม่หนู','แม่พม','สาว','หญิง']
    myself = ['ผมเป็น','ผมเอง','กระผม','พมเป็น','พมเอง','กระพม','หนูเอง','หนู','ดิฉัน','ตัวเอง','ก้อน','คลำ']
    # ตรวจสอบคำในความคิดเห็น
    for keyword in  other:
        if keyword in comment and "อาการ" not in comment :
              return 'เล่าประสบการณ์คนอื่น'
    for keyword in  myself:
        if keyword in comment:
            return 'เล่าประสบการณ์ตัวเอง'
    # หากไม่พบคำที่บ่งบอกถึงคน
    return 'ไม่ได้เล่าประสบการณ์'

In [ ]:
def detect_gender_other(comment):
    # คำที่บ่งบอกถึงเพศชาย
    male_keywords = ['พ่อ','บิดา','พี่ชาย','น้องชาย','ลูกชาย','สามี','พัว','ผัว','ปู่','ตา','คุณปู่','คุณตา','คุณพ่อ',
                     'ปู่ทวด','ตาทวด','ลุง','อาหนู','คุณอา','คุณลุง','หลายชาย','ลูกเขย','เขย','พี่เขย','น้องเขย',
                     'พ่อตา','พ่อผม','พ่อหนู','พ่อพม','ชาย','หนุ่ม']
    # คำที่บ่งบอกถึงเพศหญิง
    female_keywords = ['แม่','เเม่','คุณแม่','มารดา','พี่สาว','น้องสาว','ลูกสาว','ภรรยา','เมีย','ย่า','ยาย','คุณย่า','คุณยาย','คุณเเม่','ย่าทวด','ยายทวด','ป้า',
                       'น้า','คุณป้า','คุณน้า','หลายสาว','ลูกสะใถ้','ลูกสะใภ้','สะใภ้','พี่สะใภ้','น้องสะใภ้','เเม่ผม','เเม่หนู','เเม่พม','แม่ผม','แม่หนู','แม่พม','สาว','หญิง']
    # ตรวจสอบคำในความคิดเห็น
    for keyword in male_keywords:
        if keyword in comment and "อาการ" not in comment :
              return "เพศชาย"
    for keyword in female_keywords:
        if keyword in comment:
            return "เพศหญิง"
    # หากไม่พบคำที่บ่งบอกถึงเพศ
    return "ไม่ระบุเพศ"

In [ ]:
def detect_gender_self(comment):
    # คำที่บ่งบอกถึงเพศชาย
    male_keywords = ['ผมเป็น','ครับ','ผมเอง','คับ','กระผม','พมเป็น','พมเอง','กระพม']
    # คำที่บ่งบอกถึงเพศหญิง
    female_keywords = ['ค่ะ','คะ','หนูเอง','หนูเป็น','ดิฉัน','ก้อน','คลำ']
    # ตรวจสอบคำในความคิดเห็น
    for keyword in male_keywords:
        if keyword in comment:
            return "เพศชาย"
    for keyword in female_keywords:
        if keyword in comment:
            return "เพศหญิง"
    # หากไม่พบคำที่บ่งบอกถึงเพศ
    return "ไม่ระบุเพศ"

In [ ]:
k1=[]
k2=[]
for i in comment['คำพูดโรค']:
  k1.append(detect_person(str(i)))
  if detect_person(str(i)) == 'เล่าประสบการณ์คนอื่น':
    k2.append(detect_gender_other(str(i)))
  elif detect_person(str(i)) == 'เล่าประสบการณ์ตัวเอง':
    k2.append(detect_gender_self(str(i)))
  elif detect_person(str(i)) == 'ไม่ได้เล่าประสบการณ์':
    k2.append(detect_gender_self(str(i)))
r['ใครเล่า'] = k1
r['เพศเเบ่งโดยใช้_python'] = k2

In [ ]:
r['เพศเเบ่งโดยใช้_nlp'] = new_colgenden
r['โรค'] = new_colcan

In [ ]:
r

In [ ]:
comment = r[r['คำพูดโรค']!=0]

## อาการ

In [ ]:
print('Enter your symptoms went:')
x = input()
x = x.split(',')
symptoms = {
    "อ่อนเพลีย":["อ่อนเพลีย"],
    'เหนื่อยง่าย':['เหนื่อยง่าย','เหนื่อย','เพลีย','เหนื่อยหอบ'],
    'คันเนื้อคันตัว':['คันเนื้อคันตัว','คัน','คันตัว','คันตามตัว'],
    'มีอาการร้อน':['ร้อน','อาการรัอน','ตัวร้อน'],
    'อาการเจ็บปวด':['เจ็บปวด','เจ็บ','อาการเจ็บปวด'],
    'อาการชา':['อาการชา','ชา'],
    "เบื่ออาหาร":["เบื่ออาหาร"],
    "น้ำหนักลดโดยไม่ทราบสาเหตุ":["น้ำหนักลดโดยไม่ทราบสาเหตุ",'น้ำหนักลด','น้ำหนักลดลงเร็วมาก'],
    "ไข้เรื้อรัง":["ไข้เรื้อรัง",'ไข้','เป็นไข้'],
    "เหงื่อออกตอนกลางคืน":["เหงื่อออกตอนกลางคืน","เหงื่อออกตอนดึก",'เหงื่อออก'],
    "ปวดเมื่อย":["ปวดเมื่อยตามร่างกาย","เมื่อย"],
    "บวมหรือต่อมน้ำเหลืองโต":["บวมหรือต่อมน้ำเหลืองโต","ต่อมน้ำเหลืองโต","บวมน้ำเหลืองโต"],
    "ปวดท้อง":["ปวดท้อง"],
    "เจ็บคอ":["เจ็บคอ"],
    'ปวดใต้รักแร้':['ปวดใต้รักแร้','ปวดรักแร้'],
    'ปวดเอว':['ปวดเอว'],
    'ผิวคล้ำผิดปกติ':['ผิวคล้ำ'],
    'เจ็บเต้านม':['เจ็บเต้านม','เจ็บเต้า','เจ็บนม'],
    "ท้องอืด":["ท้องอืด",'อาการอืดท้อง','อืด'],
    "ท้องผูกเรื้อรัง":["ท้องผูกเรื้อรัง","ท้องผูก"],
    "อาหารไม่ย่อย":["อาหารไม่ย่อย"],
    "กลืนลำบาก":["กลืนลำบาก","กลืนอาหารลำบาก","กลืนยาก"],
    "ปวดท้องเรื้อรัง":["ปวดท้องเรื้อรัง"],
    "อาเจียนอาจมีเลือด":["อาเจียนเป็นเลือด","อาเจียน",'อ้วก'],
    "อุจจาระอาจมีเลือด":["อุจจาระเป็นเลือด","อึเป็นเลือด","ขี้เป็นเลือด",'อุจจาระ','ขี้','ถ่าย','ถ่ายเป็นเลือด'],
    "ไอเรื้อรัง":["ไอเรื้อรัง",'ไอ','ไอต่อเนื่อง','ไอแห้ง'],
    "ไอเป็นเลือด":["ไอเป็นเลือด"],
    "เสียงแหบ":["เสียงแหบ"],
    "หายใจลำบาก":["หายใจลำบาก","หายใจไม่อิ่ม",'หายใจ'],
    "เจ็บหรือเเน่นหน้าอก":["เจ็บหน้าอก",'เเน่นหน้าอก'],
    "เลือดออกผิดปกติจากช่องคลอด":["เลือดออกผิดปกติจากช่องคลอด","เลือดออกจากช่องคลอด","เลือดออกผิดปกติจากอวัยวะเพศ","เลือดออกจากอวัยวะเพศ",'เลือดไหล'],
    "ตกขาวผิดปกติ":["ตกขาวผิดปกติ","ตกขาว"],
    "ปัสสาวะบ่อย":["ปัสสาวะบ่อย"],
    "ปัสสาวะแสบขัด":["ปัสสาวะแสบขัด",'ฉี่แสบขัด'],
    "ปัสสาวะเป็นเลือด":["ปัสสาวะเป็นเลือด",'ฉี่เป็นเลือด'],
    "อัณฑะโตผิดปกติ":["อัณฑะโตผิดปกติ","ไข่โตผิดปกติ","ไข่ผิดปกติ",'อัณฑะผิดปกติ','อัณฑะ'],
    "แผลเรื้อรัง":["แผลเรื้อรัง","เป็นแผล","แผล"],
    "ไฝหรือปานเปลี่ยนแปลง":["ไฝหรือปานเปลี่ยนแปลง","ไฝเปลี่ยนแปลง","ปานเปลี่ยนแปลง"],
    "มีตุ่มหรือก้อนใต้ผิวหนัง":["มีตุ่มหรือก้อนใต้ผิวหนัง","มีตุ่มใต้ผิวหนัง","มีก้อนใต้ผิวหนัง",'ก้อน','ตุ่ม','มีก้อน','มีตุ่ม','ตื่ง','มีตื่ง'],
    "ปวดศีรษะ":["ปวดศีรษะ","ปวดหัว"],
    "มองเห็นภาพผิดปกติ":["มองเห็นภาพผิดปกติ","มองไม่เห็น"],
    "ชัก":["ชัก"],
}#จินีช่วยสร้าง list python อาการของโรคมะเร็งให้หน่อย+เอาอาการเพิ่มเติมด้วยเเบบไม่ซ้ำกับอันเมื่อกี้+หาเพิ่มเติมจากcomment
for i in range(len(x)):
  symptoms[f'{x[i].strip()}']=[x[i].strip()]

In [ ]:
symptoms_colcan = []
for i in range(len(list_token)):
  list_symptoms= []
  for j in range(len(list_token[i])):
    for k in symptoms:
      for l in range(len(symptoms[k])):
        if list_token[i][j] == symptoms[k][l]:
          list_symptoms.append(symptoms[k][0])
  unique_list_symptoms = list(OrderedDict.fromkeys(list_symptoms))
  if len(unique_list_symptoms) > 0:
    symptoms_colcan.append(unique_list_symptoms)
  else :
    symptoms_colcan.append(['ไม่มีการระบุอาการ'])

In [ ]:
r=comment

In [ ]:
r['อาการ']=symptoms_colcan
r

In [ ]:
label_symptoms=r['อาการ'].str.join(sep='*').str.get_dummies(sep='*')
r.join(label_symptoms)

In [ ]:
r.join(label_symptoms)

##sentence_transformers and clustering เเบ่งว่าอันไหนมี ประโยชน์กับไม่มีประโยชน์

In [ ]:
from sentence_transformers import SentenceTransformer,util
sentences = list(comment['คำพูดโรค'])
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings = model.encode(sentences)
#Normalize the embeddings to unit length
Normalize_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

In [ ]:
clustering_model = KMeans(n_clusters=2)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment)
print(len(cluster_assignment))

In [ ]:
clusternd_sentences= {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
  if cluster_id not in clusternd_sentences:
    clusternd_sentences[cluster_id] = []

  clusternd_sentences[cluster_id].append(sentences[sentence_id])

In [ ]:
len(clusternd_sentences)

In [ ]:
clusternd_sentences[1][0]

In [ ]:
len_1=[]
len_2=[]
for i in range(len(clusternd_sentences)):
    x=len(clusternd_sentences[1][i])
    len_1.append(x)
    y=len(clusternd_sentences[0][i])
    len_2.append(y)
if max(len_1) < max(len_2) :
  Pop=comment[comment['คำพูดโรค'].isin(clusternd_sentences[1])]
  Pop['โรค_clusternd'] = 'ไม่มีประโยชน์_หรือ_ให้ข้อมูลน้อยเกินไป'
  pop_use=comment[comment['คำพูดโรค'].isin(clusternd_sentences[0])]
  useful = clusternd_sentences[0]
else :
  Pop=comment[comment['คำพูดโรค'].isin(clusternd_sentences[0])]
  pop_use=comment[comment['คำพูดโรค'].isin(clusternd_sentences[1])]
  Pop['โรค_clusternd'] = 'ไม่มีประโยชน์_หรือ_ให้ข้อมูลน้อยเกินไป'
  useful = clusternd_sentences[1]

In [ ]:
Pop

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
useful_embeddings = model.encode(useful)
#Normalize the embeddings to unit length
Normalize_useful_embeddings = useful_embeddings / np.linalg.norm(useful_embeddings, axis=1, keepdims=True)
clustering_useful_model = KMeans(n_clusters=6)
clustering_useful_model.fit(Normalize_useful_embeddings)
cluster_assignment_useful = clustering_useful_model.labels_

print(cluster_assignment_useful)
print(len(cluster_assignment_useful))

In [ ]:
k_level = []
for k in range(2, len(set(pop_use['โรค']))):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(Normalize_useful_embeddings)
    labels = kmeans.labels_

    # คำนวณ Silhouette Coefficient
    silhouette_avg = silhouette_score(Normalize_useful_embeddings, labels)
    k_level.append(silhouette_avg)
print(max(k_level))

In [ ]:
import statistics
max(k_level)

In [ ]:
k_level

In [ ]:
clusternd_useful_sentences= {}
for sentence_ID, cluster_ID in enumerate(cluster_assignment_useful):
  if cluster_ID not in clusternd_useful_sentences:
    clusternd_useful_sentences[cluster_ID] = []

  clusternd_useful_sentences[cluster_ID].append(useful[sentence_ID])

In [ ]:
comment[comment['คำพูดโรค'].isin(clusternd_useful_sentences[5])]

In [ ]:
def define_Cencer_with_clusternd(use_clusternd_sentences,n_clusters):
  data = pd.DataFrame()
  for i in range(n_clusters):
    point = []
    define_C =[]
    test = comment[comment['คำพูดโรค'].isin(use_clusternd_sentences[i])]
    test_cer = list(test['โรค'])
    for j in range(len(test_cer)):
      if test_cer[j] in name_cancar:
        point.append(test_cer[j])
    if len(set(point)) == 0 :
      for x in range(len(test_cer)):
        define_C.append('ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น')
    elif len(set(point)) == 1 :
      for x in range(len(test_cer)):
        define_C.append(point[0])
    elif len(set(point)) > 1:
      for x in range(len(test_cer)):
        define_C.append('มีโอกาสเป็นโรคมะเร็งมากกว่า 2 เเบบ')
    test['โรค_clusternd'] = define_C
    data = pd.concat([data,test])
  return data

In [ ]:
define_Cencer_a=define_Cencer_with_clusternd(clusternd_useful_sentences,len(name_cancar))

In [ ]:
define_Cencer_a

In [ ]:
usedata=pd.concat([Pop,define_Cencer_a])

In [ ]:
usedata

In [ ]:
data = ['A', 'B', 'C', 'A', 'B']

# แปลง list เป็น DataFrame
df = pd.DataFrame(data, columns=['data'])

# One-hot encoding
dummy_variables = pd.get_dummies(df['data'])

# แสดงผล

In [ ]:
dummy_variables

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# ตัวอย่าง list
data = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]]

# แปลง list เป็น DataFrame
df = pd.DataFrame(data, columns=['x', 'y'])

# ลองใช้ค่า k ที่แตกต่างกัน
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(df[['x', 'y']])
    labels = kmeans.labels_

    # คำนวณ Silhouette Coefficient
    silhouette_avg = silhouette_score(df[['x', 'y']], labels)
    print(f"k = {k}, Silhouette Coefficient = {silhouette_avg}")

#REDDIT

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Carbon/data/data_commentsred_docter.csv')

In [4]:
data

,name,comments,count
0,dirkwoods,Wow. Sorry to hear about your bad news. I am a...,1456
1,Aware-Locksmith-7313,Dirkwoods … Your observations are quite inform...,237
2,puffedovenpancake,I just wanted to pop in and send some calming ...,286
3,gemurmel,"I'm so sorry to hear this. For me, it was the ...",521
4,beeboobum,Stage 4 ovarian here. Please find a palliative...,455
5,PenExactly,Inappropriate for a nurse to tell you that. Do...,148
6,Aware-Locksmith-7313,"Regret you’ve had such spread, and am wonderin...",170
7,Stickyduck468,Well that is a real kick in the teeth. So sorr...,82
8,Wyde1340,"I'm sorry to hear this, but here's some hope: ...",261
9,BitsiBones,"I think I should clarify, in the UK cancer pat...",426


## token

In [5]:
# extracting tokens
import nltk
nltk.download(['punkt', 'wordnet', 'omw-1.4', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from textblob import TextBlob

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def token(data):
  tokens = nltk.word_tokenize(data)
  tokens.insert(0, 'start')
  tokens.append('end')
  return tokens

In [7]:
def remove_(tokens):
  final = [word.lower()
         for word in tokens if word not in stopwords.words("english")]
  return final

In [8]:
# Lemmatizing
def lemma(final):
  # initialize an empty string
  str1 = ' '.join(final)
  s = TextBlob(str1)
  lemmatized_sentence = " ".join([w.lemmatize() for w in s.words])
  return final

In [9]:
data_com = data['comments'].to_list()

In [10]:
list_token_red = []
for item_comment_red in range(len(data_com)):
  data_r_token = token(data_com[item_comment_red].translate(str.maketrans('', '', string.punctuation)))
  data_r_token = remove_(data_r_token)
  data_r_token = lemma(data_r_token)
  list_token_red.append(data_r_token)
print(list_token_red)

[['start', 'wow', 'sorry', 'hear', 'bad', 'news', 'i', 'retired', 'physician', 'i', 'know', 'details', 'case', 'i', 'suspect', 'miscommunication', 'kind', 'do', 'accept', 'nurses', 'assessment', 'situation', 'final', 'word', 'if', 'mychart', 'i', 'would', 'reach', 'oncologist', 'ask', 'terms', 'possible', 'miscommunication', 'possibly', 'confused', 'nurse', 'said', 'you', 'also', 'given', 'chance', 'speak', 'radiation', 'oncologist', 'expert', 'friday', 'appointment', 'i', 'would', 'place', 'far', 'faith', 'radiation', 'oncologist', 'says', 'i', 'spinal', 'lesions', 'causing', 'sciatica', 'start', 'motor', 'weakness', 'i', 'received', 'radiation', 'spine', 'reversed', 'pain', 'motor', 'strength', 'returned', 'normal', 'it', 'seems', 'told', 'might', 'paralyzed', 'lesion', 'someone', 'done', 'motor', 'exam', 'formally', 'tested', 'strength', 'right', 'as', 'reaching', 'oncologist', 'pcp', 'might', 'ask', 'anyone', 'concerned', 'enough', 'motor', 'exam', 'placed', 'steroids', 'awaiting',

##เเบ่งโรค

In [11]:
cancer_names_en = [
    "Bladder cancer",
    "Endometrial cancer",
    "Cervical cancer",
    "Leukemia",
    "Liver cancer",
    "Pancreatic cancer",
    "Prostate cancer",
    "Stomach cancer",
    "Esophageal cancer",
    "Breast cancer",
    "Lung cancer",
    "Colorectal cancer",
    "Ovarian cancer",
    "Oral cancer",
    "Thyroid cancer",
    "Lymphoma",
    "Brain cancer",
    "Skin cancer",
    "Intestinal cancer",  # More general term
    "Bile duct cancer",
    "Colorectal cancer",  # More specific term
    "Melanoma",
    "Hodgkin lymphoma",
    "Acute leukemia",
    "Bone cancer",
    "Central nervous system cancer",
    "Kidney cancer",
    "Uterine cancer",  # More general term
    "Gallbladder cancer",
    "Spinal cord cancer",
    "Eye cancer",
    "Vaginal cancer",
    "Penile cancer",
    "Vulvar cancer",
    "Adrenal gland cancer",
    "Salivary gland cancer",
    "Ovarian cyst cancer",
]

In [12]:
column_cancer_nlp_rad=[]
for list_token_i in range(len(list_token_red)):#len(comment)
  list_cancer_en = []
  for list_token_k in range(len(list_token_red[list_token_i])):
    for cancer_names_j in range(len(cancer_names_en)):
      if (list_token_red[list_token_i][list_token_k] == 'cancer'):
        list_cancer_en.append(list_token_red[list_token_i][list_token_k-1]+list_token_red[list_token_i][list_token_k])
        unique_list_en = list(OrderedDict.fromkeys(list_cancer_en))
#----------------------------------------------------------------------------
  list_define_cancer_en = []
  new_list_en=[]
  for i in range(len(unique_list_en)):
    for j in range(len(cancer_names_en)):
      if unique_list_en[i]==cancer_names_en[j]:
        list_define_cancer_en.append(unique_list[i])
#----------------------------------------------------------------------------
  cancer_list_de_red =[]
  if len(list_cancer_en) > 0:
    if len(list_cancer_en) == 2:
      cancer_list_de_red.append('Tell more than 2 diseases.')
    elif len(list_cancer_en)==1:
      cancer_list_de_red.append(list_cancer_en[0])
  elif list_cancer_en==[]:
    cancer_list_de_red.append('Unable to identify / not sure if it is')
  if len(cancer_list_de_red)> 0 :
    column_cancer_nlp_rad.append(cancer_list_de_red[0])
  elif len(cancer_list_de_red)== 0 :
    column_cancer_nlp_rad.append('Unable to identify / not sure if it is')

In [13]:
list_define_cancer_en

[]

In [14]:
column_cancer_nlp_rad

['Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is',
 'Unable to identify / not sure if it is']

##เเบ่งว่ามีประโยนช์หรือไม่มีประโยน์

In [15]:
len(data['comments'])

22

In [24]:
from sentence_transformers import SentenceTransformer,util
sentences_en = list(data['comments'])
model_red = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings_red = model_red.encode(sentences_en)
#Normalize the embeddings to unit length
Normalize_embeddings_red = embeddings_red / np.linalg.norm(embeddings_red, axis=1, keepdims=True)
clustering_model_red = KMeans(n_clusters=2)
clustering_model_red.fit(Normalize_embeddings_red)
cluster_assignment_en = clustering_model_red.labels_
clusternd_sentences_en= {}
for sentence_id_red, cluster_id_red in enumerate(cluster_assignment_en):
  if cluster_id_red not in clusternd_sentences_en:
    clusternd_sentences_en[cluster_id_red] = []

  clusternd_sentences_en[cluster_id_red].append(sentences_en[sentence_id_red])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [29]:
clusternd_sentences_en

{1: ["Wow. Sorry to hear about your bad news. I am a retired physician. I do not know the details of your case but I suspect that there was some miscommunication of some kind. Do not accept the nurse's assessment of your situation as the final word. If you have MyChart I would reach out to the Oncologist and ask about this in terms of possible miscommunication or possibly being confused about what the nurse said. You should also be given the chance to speak with the Radiation Oncologist as the expert on Friday at your appointment. I would place far more faith in what the Radiation Oncologist says. I had spinal lesions causing sciatica and had the start of motor weakness. I received radiation to the spine which reversed the pain and my motor strength has returned to normal. It seems to me if you have been told you might be paralyzed by this lesion that someone should have done a motor exam on you (formally tested your strength right now). As you are reaching out to the Oncologist and yo

In [26]:
print(cluster_assignment_en)
print(len(cluster_assignment_en))

[1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 0 1 1 1]
22


In [31]:
max(len_3)

1407

In [32]:
len_3=[]
len_4=[]
for i in range(len(clusternd_sentences_en)):
    x=len(clusternd_sentences_en[1][i])
    len_3.append(x)
    y=len(clusternd_sentences_en[0][i])
    len_4.append(y)
if max(len_3) < max(len_4) :
  Pop_en=data[data['comments'].isin(clusternd_sentences_en[1])]
  Pop_en['cancer_clusternd'] = 'Not useful or not giving too much information'
  Pop_en[defind_cancer_with_clusternd] = 'Not useful or not giving too much information'
  Pop_en_use=data[data['comments'].isin(clusternd_sentences_en[0])]
  useful = clusternd_sentences_en[0]
  # useful['cancer_clusternd'] = 'It may be useful or it may provide enough information.'
else :
  Pop_en=data[data['comments'].isin(clusternd_sentences_en[0])]
  Pop_en_use=data[data['comments'].isin(clusternd_sentences_en[1])]
  Pop_en['cancer_clusternd'] = 'Not useful or not giving too much information'
  Pop_en[defind_cancer_with_clusternd] = 'Not useful or not giving too much information'
  useful = clusternd_sentences_en[1]

<ipython-input-32-07b3d1a808b7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pop_en['cancer_clusternd'] = 'Not useful or not giving too much information'


In [34]:
Pop_en

,name,comments,count,cancer_clusternd
7,Stickyduck468,Well that is a real kick in the teeth. So sorr...,82,Not useful or not giving too much information
10,skelterjohn,"That's really rough, sorry to hear about that ...",64,Not useful or not giving too much information
16,pthrizzle,I’m so sorry. Sending prayers up and hopeful t...,73,Not useful or not giving too much information
18,mmmonicapb,Im so sorry you received this tough news today...,196,Not useful or not giving too much information


In [36]:
model_en_useful = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
useful_embeddings_en = model_en_useful.encode(useful)
#Normalize the embeddings to unit length
Normalize_useful_embeddings_en = useful_embeddings_en / np.linalg.norm(useful_embeddings_en, axis=1, keepdims=True)
clustering_useful_model_en = KMeans(n_clusters=3)
clustering_useful_model_en.fit(Normalize_useful_embeddings_en)
cluster_assignment_useful_en = clustering_useful_model_en.labels_
clusternd_useful_sentences_en= {}
for cluster_ID_en in range(clustering_useful_model_en.n_clusters):
    clusternd_useful_sentences_en[cluster_ID_en] = []

for sentence_ID_en, cluster_ID_en in enumerate(cluster_assignment_useful_en):
    clusternd_useful_sentences_en[cluster_ID_en].append(useful[sentence_ID_en])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [37]:
all_reddit_data = data
all_reddit_data['defind_cancer_with_nlp'] = column_cancer_nlp_rad

In [38]:
def define_Cencer_with_clusternd(use_clusternd_sentences,n_clusters):
  data = pd.DataFrame()
  for i in range(n_clusters):
    point = []
    define_C =[]
    test = all_reddit_data[all_reddit_data['comments'].isin(use_clusternd_sentences[i])]
    test_cer = list(test['defind_cancer_with_nlp'])
    for j in range(len(test_cer)):
      if test_cer[j] in cancer_names_en:
        point.append(test_cer[j])
    if len(set(point)) == 0 :
      for x in range(len(test_cer)):
        define_C.append('Unable to identify/not sure if it is')
    elif len(set(point)) == 1 :
      for x in range(len(test_cer)):
        define_C.append(point[0])
    elif len(set(point)) > 1:
      for x in range(len(test_cer)):
        define_C.append('There is a chance of having more than 2 types of cancer')
    test['defind_cancer_with_clusternd'] = define_C
    data = pd.concat([data,test])
  return data

In [39]:
define_Cencer_b = define_Cencer_with_clusternd(clusternd_useful_sentences_en,3)

<ipython-input-38-fee11a6461be>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['defind_cancer_with_clusternd'] = define_C
<ipython-input-38-fee11a6461be>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['defind_cancer_with_clusternd'] = define_C
<ipython-input-38-fee11a6461be>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [41]:
define_Cencer_b['cancer_clusternd'] = 'It may be useful or it may provide enough information.'

In [54]:
usedata=pd.concat([Pop_en,define_Cencer_b])

In [57]:
usedata

,name,comments,count,cancer_clusternd,defind_cancer_with_nlp,defind_cancer_with_clusternd,defind_Genden_with_nlp,defind_Genden_with_python,defind_exp_with_python,symptoms_colcan_en
7,Stickyduck468,Well that is a real kick in the teeth. So sorr...,82,Not useful or not giving too much information,NaN,NaN,Unable to identify/not sure if it is,Gender not specified,Tell about your own experiences,[No symptoms identified]
10,skelterjohn,"That's really rough, sorry to hear about that ...",64,Not useful or not giving too much information,NaN,NaN,Unable to identify/not sure if it is,Male,Tell other people's experiences,[No symptoms identified]
16,pthrizzle,I’m so sorry. Sending prayers up and hopeful t...,73,Not useful or not giving too much information,NaN,NaN,Unable to identify/not sure if it is,Male,Tell other people's experiences,[No symptoms identified]
18,mmmonicapb,Im so sorry you received this tough news today...,196,Not useful or not giving too much information,NaN,NaN,Unable to identify/not sure if it is,Gender not specified,Tell about your own experiences,[No symptoms identified]
4,beeboobum,Stage 4 ovarian here. Please find a palliative...,455,It may be useful or it may provide enough info...,Unable to identify / not sure if it is,Unable to identify/not sure if it is,Unable to identify/not sure if it is,Male,Tell other people's experiences,[No symptoms identified]
21,merce007,"I'm in a similar situation, I had great succes...",1335,It may be useful or it may provide enough info...,Unable to identify / not sure if it is,Unable to identify/not sure if it is,Unable to identify/not sure if it is,Male,Tell other people's experiences,[No symptoms identified]
0,dirkwoods,Wow. Sorry to hear about your bad news. I am a...,1456,It may be useful or it may provide enough info...,Unable to identify / not sure if it is,Unable to identify/not sure if it is,Unable to identify/not sure if it is,Gender not specified,Tell other people's experiences,[No symptoms identified]
1,Aware-Locksmith-7313,Dirkwoods … Your observations are quite inform...,237,It may be useful or it may provide enough info...,Unable to identify / not sure if it is,Unable to identify/not sure if it is,Unable to identify/not sure if it is,Male,Tell other people's experiences,[No symptoms identified]
5,PenExactly,Inappropriate for a nurse to tell you that. Do...,148,It may be useful or it may provide enough info...,Unable to identify / not sure if it is,Unable to identify/not sure if it is,Unable to identify/not sure if it is,Male,Tell other people's experiences,[No symptoms identified]
6,Aware-Locksmith-7313,"Regret you’ve had such spread, and am wonderin...",170,It may be useful or it may provide enough info...,Unable to identify / not sure if it is,Unable to identify/not sure if it is,Unable to identify/not sure if it is,Male,Tell other people's experiences,[No symptoms identified]


##เเบ่งเพศ

In [43]:
Genden_en = {'Male':['dad', "father",
    "stepfather",
    "grandfather",
    "great-grandfather",
    "husband",
    "boyfriend",
    "fiancé",
    "son",
    "stepson",
    "grandson",
    "great-grandson",
    "brother",
    "half-brother",
    "stepbrother",
    "uncle",
    "great-uncle",
    "nephew",
    "great-nephew",
    "father-in-law",
    "stepfather-in-law",
    "grandfather-in-law",
    "son-in-law",
    "stepson-in-law",
    "grandson-in-law",
    "brother-in-law",
    "half-brother-in-law",
    "stepbrother-in-law",
    "uncle-in-law",
    "great-uncle-in-law",
    "nephew-in-law",
    "great-nephew-in-law",
    "godfather",
    "stepfather",
    "godson",
    "foster father",
    "foster son",
    "stepson",
    "stepbrother",
    "half-brother",
    "great-uncle",
    "great-nephew",
    "first cousin",
    "second cousin",
    "third cousin",
    "father-in-law",
    "stepfather-in-law",
    "grandfather-in-law",
    "son-in-law",
    "stepson-in-law",
    "grandson-in-law",
    "brother-in-law",
    "half-brother-in-law",
    "stepbrother-in-law",
    "uncle-in-law",
    "great-uncle-in-law",
    "nephew-in-law",
    "great-nephew-in-law"
    'he','him','his','boy']
    ,
    'Female':[
    "mother",
    'mom',
    "stepmother",
    "grandmother",
    "great-grandmother",
    "wife",
    "girlfriend",
    "fiancée",
    "daughter",
    "stepdaughter",
    "granddaughter",
    "great-granddaughter",
    "sister",
    "half-sister",
    "stepsister",
    "aunt",
    "great-aunt",
    "niece",
    "great-niece",
    "mother-in-law",
    "stepmother-in-law",
    "grandmother-in-law",
    "daughter-in-law",
    "stepdaughter-in-law",
    "granddaughter-in-law",
    "sister-in-law",
    "half-sister-in-law",
    "stepsister-in-law",
    "aunt-in-law",
    "great-aunt-in-law",
    "niece-in-law",
    "great-niece-in-law",
    "cousin-in-law",
    "godmother",
    "stepmother",
    "goddaughter",
    "foster mother",
    "foster daughter",
    "stepdaughter",
    "stepsister",
    "half-sister",
    "great-aunt",
    "great-niece",
    "mother-in-law",
    "stepmother-in-law",
    "grandmother-in-law",
    "daughter-in-law",
    "stepdaughter-in-law",
    "granddaughter-in-law",
    "sister-in-law",
    "half-sister-in-law",
    "stepsister-in-law",
    "aunt-in-law",
    "great-aunt-in-law",
    "niece-in-law",
    "great-niece-in-law",
    'she','her','hers','mommy','girl'
]}

In [44]:
new_colgenden_en=[]
list_genden_en=[]
for i in range(len(list_token_red)):
  for j in range(len(Genden_en['Female'])):
    for k in range(len(list_token_red[i])):
      if (list_token_red[i][k] == Genden_en['Male'][j]):
        list_genden_en.append('Male')
      elif(list_token_red[i][k] == Genden_en['Female'][j]):
        list_genden_en.append('Female')
  genden_list_en =[]
  genden_list_en = list(OrderedDict.fromkeys(list_genden_en)) # ลบคำซ้ำ
  #-------------------------------------------------------------------
  list_define_genden_en = []
  if len(genden_list_en) > 0 :
    if len(genden_list_en) == 2:
      list_define_genden_en.append('Both genders are told.')
    elif len(genden_list_en)==1:
      list_define_genden_en.append(genden_list_en[0])
  elif len(genden_list_en)==0:
    list_define_genden_en.append('Unable to identify/not sure if it is')
  genden_list_de_en =[]
  genden_list_de_en = list(OrderedDict.fromkeys(list_define_genden_en))
  new_colgenden_en.append(genden_list_de_en[0])
print(new_colgenden_en)

['Unable to identify/not sure if it is', 'Unable to identify/not sure if it is', 'Unable to identify/not sure if it is', 'Unable to identify/not sure if it is', 'Unable to identify/not sure if it is', 'Unable to identify/not sure if it is', 'Unable to identify/not sure if it is', 'Unable to identify/not sure if it is', 'Unable to identify/not sure if it is', 'Unable to identify/not sure if it is', 'Unable to identify/not sure if it is', 'Unable to identify/not sure if it is', 'Unable to identify/not sure if it is', 'Unable to identify/not sure if it is', 'Male', 'Male', 'Male', 'Male', 'Male', 'Both genders are told.', 'Both genders are told.', 'Both genders are told.']


##exp reddit

In [45]:
def detect_person_en(comment):
    # คำที่ใช้ตรวจสอบว่ามีใครเป็นคนอยู่ในความคิดเห็น
    other = ['you','your','we','they','he','she','him','her','it']
    myself = ['my','myself']
    # ,"mother",'mom',"stepmother",
    # "grandmother","great-grandmother","wife","girlfriend","fiancée","daughter","stepdaughter",
    # "granddaughter","great-granddaughter","sister","half-sister","stepsister","aunt","great-aunt","niece",
    # "great-niece","mother-in-law","stepmother-in-law","grandmother-in-law","daughter-in-law","stepdaughter-in-law",
    # "granddaughter-in-law","sister-in-law","half-sister-in-law","stepsister-in-law","aunt-in-law","great-aunt-in-law",
    # "niece-in-law","great-niece-in-law","cousin-in-law","godmother","stepmother","goddaughter","foster mother","foster daughter","stepdaughter","stepsister",
    # "half-sister","great-aunt","great-niece","mother-in-law","stepmother-in-law","grandmother-in-law","daughter-in-law","stepdaughter-in-law",
    # "granddaughter-in-law","sister-in-law","half-sister-in-law","stepsister-in-law","aunt-in-law","great-aunt-in-law","niece-in-law","great-niece-in-law",'mommy',
    # 'dad', "father","stepfather","grandfather","great-grandfather","husband",
    # "boyfriend","fiancé","son","stepson","grandson","great-grandson","brother","half-brother","stepbrother","uncle","great-uncle","nephew",
    # "great-nephew","father-in-law","stepfather-in-law","grandfather-in-law","son-in-law","stepson-in-law","grandson-in-law","brother-in-law",
    # "half-brother-in-law","stepbrother-in-law","uncle-in-law","great-uncle-in-law","nephew-in-law","great-nephew-in-law","godfather","stepfather",
    # "godson","foster father","foster son","stepson","stepbrother","half-brother","great-uncle","great-nephew","first cousin","second cousin",
    # "third cousin","father-in-law","stepfather-in-law","grandfather-in-law","son-in-law","stepson-in-law","grandson-in-law","brother-in-law",
    # "half-brother-in-law","stepbrother-in-law","uncle-in-law","great-uncle-in-law","nephew-in-law",
    # "great-nephew-in-law"
    # ตรวจสอบคำในความคิดเห็น
    for keyword in  myself:
        if keyword in comment:
          return "Tell about your own experiences"
    for keyword in  other:
        if keyword in comment:
          return "Tell other people's experiences"
    # หากไม่พบคำที่บ่งบอกถึงคน
    return "Didn't tell the experience"

In [46]:
def detect_gender_other_en(comment):
    # คำที่บ่งบอกถึงเพศชาย
    male_keywords = ['he','him','dad', "father","stepfather","grandfather","great-grandfather","husband",
    "boyfriend","fiancé","son","stepson","grandson","great-grandson","brother","half-brother","stepbrother","uncle","great-uncle","nephew",
    "great-nephew","father-in-law","stepfather-in-law","grandfather-in-law","son-in-law","stepson-in-law","grandson-in-law","brother-in-law",
    "half-brother-in-law","stepbrother-in-law","uncle-in-law","great-uncle-in-law","nephew-in-law","great-nephew-in-law","godfather","stepfather",
    "godson","foster father","foster son","stepson","stepbrother","half-brother","great-uncle","great-nephew","first cousin","second cousin",
    "third cousin","father-in-law","stepfather-in-law","grandfather-in-law","son-in-law","stepson-in-law","grandson-in-law","brother-in-law",
    "half-brother-in-law","stepbrother-in-law","uncle-in-law","great-uncle-in-law","nephew-in-law",
    "great-nephew-in-law",'boy']
    # คำที่บ่งบอกถึงเพศหญิง
    female_keywords = ["mother",'mom',"stepmother",'girl',
    "grandmother","great-grandmother","wife","girlfriend","fiancée","daughter","stepdaughter",
    "granddaughter","great-granddaughter","sister","half-sister","stepsister","aunt","great-aunt","niece",
    "great-niece","mother-in-law","stepmother-in-law","grandmother-in-law","daughter-in-law","stepdaughter-in-law",
    "granddaughter-in-law","sister-in-law","half-sister-in-law","stepsister-in-law","aunt-in-law","great-aunt-in-law",
    "niece-in-law","great-niece-in-law","cousin-in-law","godmother","stepmother","goddaughter","foster mother","foster daughter","stepdaughter","stepsister",
    "half-sister","great-aunt","great-niece","mother-in-law","stepmother-in-law","grandmother-in-law","daughter-in-law","stepdaughter-in-law",
    "granddaughter-in-law","sister-in-law","half-sister-in-law","stepsister-in-law","aunt-in-law","great-aunt-in-law","niece-in-law","great-niece-in-law",'mommy','she','her']
    # ตรวจสอบคำในความคิดเห็น
    for keyword in male_keywords:
        if keyword in comment :
          print(keyword)
          return "Male"
    for keyword in female_keywords:
        if keyword in comment:
          print(keyword)
          return "Female"
    # หากไม่พบคำที่บ่งบอกถึงเพศ
    return "Gender not specified"

In [47]:
k3=[]
k4=[]
for i in data['comments']:
  k3.append(detect_person_en(str(i)))
  if detect_person_en(str(i)) == "Tell about your own experiences":
    k4.append("Gender not specified")
  elif detect_person_en(str(i)) == "Tell other people's experiences":
    k4.append(detect_gender_other_en(str(i)))
  elif detect_person_en(str(i)) == "Didn't tell the experience":
    k4.append(detect_gender_other_en(str(i)))

he
he
he
he
he
he
he
he
he
he
he
he
he
he


In [48]:
k3

['Tell about your own experiences',
 "Tell other people's experiences",
 "Tell other people's experiences",
 'Tell about your own experiences',
 "Tell other people's experiences",
 "Tell other people's experiences",
 "Tell other people's experiences",
 "Tell other people's experiences",
 "Tell other people's experiences",
 "Tell other people's experiences",
 "Tell other people's experiences",
 "Tell other people's experiences",
 "Tell other people's experiences",
 "Tell other people's experiences",
 'Tell about your own experiences',
 "Tell other people's experiences",
 "Tell other people's experiences",
 "Tell other people's experiences",
 "Tell other people's experiences",
 "Tell other people's experiences",
 'Tell about your own experiences',
 'Tell about your own experiences']

In [49]:
k4

['Gender not specified',
 'Male',
 'Male',
 'Gender not specified',
 'Male',
 'Male',
 'Gender not specified',
 'Male',
 'Male',
 'Male',
 'Male',
 'Male',
 'Male',
 'Male',
 'Gender not specified',
 'Male',
 'Gender not specified',
 'Male',
 'Gender not specified',
 'Male',
 'Gender not specified',
 'Gender not specified']

## หาอาการ

In [50]:
cancer_symptoms_en = {
    "Fatigue": ["Fatigue"],
    "Tiredness": ["Fatigue", "Tiredness", "Lethargy", "Shortness of breath"],
    "Itching": ["Itching", "Itching skin"],
    "Feverishness": ["Feverishness", "Fever"],
    "Pain": ["Pain", "Aches"],
    "Numbness": ["Numbness", "Tingling"],
    "Loss of appetite": ["Loss of appetite"],
    "Unexplained weight loss": ["Unexplained weight loss", "Weight loss", "Rapid weight loss"],
    "Chronic fever": ["Chronic fever", "Fever"],
    "Night sweats": ["Night sweats", "Sweating"],
    "Body aches": ["Body aches", "Aches"],
    "Swollen lymph nodes": ["Swollen lymph nodes"],
    "Stomach pain": ["Stomach pain"],
    "Sore throat": ["Sore throat"],
    "Swollen armpits": ["Swollen armpits"],
    "Back pain": ["Back pain"],
    "Darkened skin": ["Darkened skin"],
    "Breast pain": ["Breast pain"],
    "Bloating": ["Bloating", "Abdominal bloating", "Bloated"],
    "Chronic constipation": ["Chronic constipation", "Constipation"],
    "Indigestion": ["Indigestion"],
    "Difficulty swallowing": ["Difficulty swallowing", "Dysphagia"],
    "Chronic abdominal pain": ["Chronic abdominal pain"],
    "Vomiting blood": ["Vomiting blood", "Vomiting", "Emesis"],
    "Bloody stools": ["Bloody stools", "Blood in stool", "Melena"],
    "Chronic cough": ["Chronic cough", "Cough", "Persistent cough", "Dry cough"],
    "Coughing up blood": ["Coughing up blood"],
    "Hoarseness": ["Hoarseness"],
    "Shortness of breath": ["Shortness of breath", "Dyspnea"],
    "Chest pain": ["Chest pain"],
    "Vaginal bleeding": ["Vaginal bleeding", "Bleeding from vagina", "Abnormal bleeding"],
    "Vaginal discharge": ["Vaginal discharge"],
    "Frequent urination": ["Frequent urination"],
    "Urinary tract infection": ["Urinary tract infection", "Dysuria"],
    "Hematuria": ["Hematuria"],
    "Enlarged testicles": ["Enlarged testicles"],
    "Skin ulcers": ["Skin ulcers", "Ulcers"],
    "Changing moles or freckles": ["Changing moles or freckles"],
    "Lumps or bumps under the skin": ["Lumps or bumps under the skin"],
    "Headache": ["Headache"],
    "Vision problems": ["Vision problems"],
    "Seizures": ["Seizures"],
}

In [51]:
symptoms_colcan_en = []
for i in range(len(list_token_red)):
  list_symptoms_en= []
  for j in range(len(list_token_red[i])):
    for k in cancer_symptoms_en:
      for l in range(len(cancer_symptoms_en[k])):
        if list_token_red[i][j] == cancer_symptoms_en[k][l]:
          list_symptoms_en.append(cancer_symptoms_en[k][0])
  unique_list_symptoms_en = list(OrderedDict.fromkeys(list_symptoms_en))
  if len(unique_list_symptoms_en) > 0:
    symptoms_colcan_en.append(unique_list_symptoms_en)
  else :
    symptoms_colcan_en.append(['No symptoms identified'])

In [52]:
list_symptoms_en

[]

##รวมตาราง

In [56]:
all_reddit_data = usedata
all_reddit_data['defind_Genden_with_nlp'] = new_colgenden_en
all_reddit_data['defind_Genden_with_python'] = k4
all_reddit_data['defind_exp_with_python'] = k3
all_reddit_data['symptoms_colcan_en'] = symptoms_colcan_en
label_symptoms_en=all_reddit_data['symptoms_colcan_en'].str.join(sep='*').str.get_dummies(sep='*')
all_reddit_data.join(label_symptoms_en)
all_reddit_data

,name,comments,count,cancer_clusternd,defind_cancer_with_nlp,defind_cancer_with_clusternd,defind_Genden_with_nlp,defind_Genden_with_python,defind_exp_with_python,symptoms_colcan_en
7,Stickyduck468,Well that is a real kick in the teeth. So sorr...,82,Not useful or not giving too much information,NaN,NaN,Unable to identify/not sure if it is,Gender not specified,Tell about your own experiences,[No symptoms identified]
10,skelterjohn,"That's really rough, sorry to hear about that ...",64,Not useful or not giving too much information,NaN,NaN,Unable to identify/not sure if it is,Male,Tell other people's experiences,[No symptoms identified]
16,pthrizzle,I’m so sorry. Sending prayers up and hopeful t...,73,Not useful or not giving too much information,NaN,NaN,Unable to identify/not sure if it is,Male,Tell other people's experiences,[No symptoms identified]
18,mmmonicapb,Im so sorry you received this tough news today...,196,Not useful or not giving too much information,NaN,NaN,Unable to identify/not sure if it is,Gender not specified,Tell about your own experiences,[No symptoms identified]
4,beeboobum,Stage 4 ovarian here. Please find a palliative...,455,It may be useful or it may provide enough info...,Unable to identify / not sure if it is,Unable to identify/not sure if it is,Unable to identify/not sure if it is,Male,Tell other people's experiences,[No symptoms identified]
21,merce007,"I'm in a similar situation, I had great succes...",1335,It may be useful or it may provide enough info...,Unable to identify / not sure if it is,Unable to identify/not sure if it is,Unable to identify/not sure if it is,Male,Tell other people's experiences,[No symptoms identified]
0,dirkwoods,Wow. Sorry to hear about your bad news. I am a...,1456,It may be useful or it may provide enough info...,Unable to identify / not sure if it is,Unable to identify/not sure if it is,Unable to identify/not sure if it is,Gender not specified,Tell other people's experiences,[No symptoms identified]
1,Aware-Locksmith-7313,Dirkwoods … Your observations are quite inform...,237,It may be useful or it may provide enough info...,Unable to identify / not sure if it is,Unable to identify/not sure if it is,Unable to identify/not sure if it is,Male,Tell other people's experiences,[No symptoms identified]
5,PenExactly,Inappropriate for a nurse to tell you that. Do...,148,It may be useful or it may provide enough info...,Unable to identify / not sure if it is,Unable to identify/not sure if it is,Unable to identify/not sure if it is,Male,Tell other people's experiences,[No symptoms identified]
6,Aware-Locksmith-7313,"Regret you’ve had such spread, and am wonderin...",170,It may be useful or it may provide enough info...,Unable to identify / not sure if it is,Unable to identify/not sure if it is,Unable to identify/not sure if it is,Male,Tell other people's experiences,[No symptoms identified]
